## 7. ONNX模型导出为KModel格式

### 7.1 ONNX模型直接转换为KModel格式

In [4]:
import nncase
import onnxsim
import os
import onnx

def parse_model_input_output(model_file):
    onnx_model = onnx.load(model_file)
    input_all = [node.name for node in onnx_model.graph.input]
    input_initializer = [node.name for node in onnx_model.graph.initializer]
    input_names = list(set(input_all) - set(input_initializer))
    input_tensors = [node for node in onnx_model.graph.input if node.name in input_names]

    # input
    inputs = []
    for _, e in enumerate(input_tensors):
        onnx_type = e.type.tensor_type
        input_dict = {}
        input_dict['name'] = e.name
        input_dict['dtype'] = onnx.mapping.TENSOR_TYPE_TO_NP_TYPE[onnx_type.elem_type]
        input_dict['shape'] = [(i.dim_value if i.dim_value != 0 else d) for i, d in zip(
            onnx_type.shape.dim, [1, 3, 224, 224])]
        inputs.append(input_dict)

    return onnx_model, inputs

def onnx_simplify(model_file):
    onnx_model, inputs = parse_model_input_output(model_file)
    onnx_model = onnx.shape_inference.infer_shapes(onnx_model)
    input_shapes = {}
    for input in inputs:
        input_shapes[input['name']] = input['shape']

    onnx_model, check = onnxsim.simplify(onnx_model, overwrite_input_shapes=input_shapes)
    assert check, "Simplified ONNX model could not be validated"

    model_file = os.path.join(os.path.dirname(model_file), 'yolo_free_large_simplified.onnx')
    onnx.save_model(onnx_model, model_file)
    return model_file

def read_model_file(model_file):
    with open(model_file, 'rb') as f:
        model_content = f.read()
    return model_content

# onnx simplify
model_file = onnx_simplify("../weights/onnx/11/yolo_free_large.onnx")

# compile_options
compile_options = nncase.CompileOptions()
compile_options.target = 'k210'
compile_options.dump_ir = True
compile_options.dump_asm = True
compile_options.dump_dir = 'tmp'

# compiler
compiler = nncase.Compiler(compile_options)

# import_options
import_options = nncase.ImportOptions()

# import
model_content = read_model_file(model_file)
compiler.import_onnx(model_content, import_options)

# compile
compiler.compile()

# kmodel
kmodel = compiler.gencode_tobytes()
name = os.path.basename(model_file).split(".")[0]
with open(f'{name}.kmodel', 'wb') as f:
    f.write(kmodel)

/tmp/ipykernel_1940903/2480207409.py:19: DeprecationWarning: `mapping.TENSOR_TYPE_TO_NP_TYPE` is now deprecated and will be removed in a future release.To silence this warning, please use `helper.tensor_dtype_to_np_dtype` instead.
  input_dict['dtype'] = onnx.mapping.TENSOR_TYPE_TO_NP_TYPE[onnx_type.elem_type]


1. Import graph...


RuntimeError: Not supported ONNX opcode: NonZero

## 7.2 ONNX模型转换成TFlite格式再转换成KModel格式

In [8]:
from onnx_tf.backend import prepare
import tensorflow as tf

TF_PATH = "../weights/tflite/yolo_free_large_simplified.pb" # where the representation of tensorflow model will be stored
ONNX_PATH = "../weights/onnx/11/yolo_free_large_simplified.onnx" # path to my existing ONNX model
onnx_model = onnx.load(ONNX_PATH)  # load onnx model
tf_rep = prepare(onnx_model)  # creating TensorflowRep object
tf_rep.export_graph(TF_PATH)

TFLITE_PATH = "../weights/tflite/yolo_free_large_simplified.tflite"
converter = tf.lite.TFLiteConverter.from_saved_model(TF_PATH)
## 启用量化配置
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tf_lite_model = converter.convert()
with open(TFLITE_PATH, 'wb') as f:
    f.write(tf_lite_model)

/home/xd324/anaconda3/envs/pt20/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
2023-05-07 13:51:19.196991: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-07 13:51:19.197144: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node 

INFO:tensorflow:Assets written to: ../weights/tflite/yolo_free_large_simplified.pb/assets


INFO:tensorflow:Assets written to: ../weights/tflite/yolo_free_large_simplified.pb/assets
2023-05-07 13:51:49.693189: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'serving_default_images' with dtype float and shape [1,3,640,640]
	 [[{{node serving_default_images}}]]
2023-05-07 13:51:49.742560: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-05-07 13:51:49.742581: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-05-07 13:51:49.743044: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ../weights/tflite/yolo_free_large_simplified.pb
2023-05-07 13:51:49.762304: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-07 13:51:49.762333: I tens

ConverterError: <unknown>:0: error: loc(callsite(callsite(fused["Cast:", "onnx_tf_prefix_/Div/Cast@__inference___call___1379"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper_1524"]) at fused["StatefulPartitionedCall:", "StatefulPartitionedCall"])): 'tf.Cast' op is neither a custom op nor a flex op
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall"]): called from
<unknown>:0: note: loc(callsite(callsite(fused["Cast:", "onnx_tf_prefix_/Div/Cast@__inference___call___1379"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper_1524"]) at fused["StatefulPartitionedCall:", "StatefulPartitionedCall"])): Error code: ERROR_NEEDS_FLEX_OPS
<unknown>:0: error: loc(callsite(callsite(fused["RealDiv:", "onnx_tf_prefix_/Div@__inference___call___1379"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper_1524"]) at fused["StatefulPartitionedCall:", "S